# Compile functions

In this part of the tutorial, we'll learn how to change pages and questions immediately before they are "compiled" and displayed to the user. Compile functions allow the survey to change based on a user's responses.

Run the cell below to create a test app.

In [ ]:
import os

from hemlock import User, Page, create_test_app
from hemlock.functional import compile
from hemlock.questions import Check, Input, Label
from sqlalchemy_mutable.utils import partial

os.environ.pop("GITPOD_HOST", None)

app = create_test_app()

Suppose we have an `Input` on page 0 asking the user what their name is. One page 1, we want to greet them by saying, "Hello, USERNAME!"

Our first thought might be to store the `Input` on page 0 in a variable `name_input`, then use the response (accessible with `name_input.response`) in a `Label` on page 1.

Run the cell below and see what happens.

In [ ]:
def seed():
    return [
        Page(
            name_input := Input("What's your name?")
        ),
        Page(
            Label(f"Hello, {name_input.response}!")
        )
    ]


user = User.make_test_user(seed)
user.test_request(["Aristotle"]).display()

Why is the name `None`? Because we used `name_input.response` *before* the user responded the question. To implement the greeting, we need to set the greeting label *after* the user tells us their name.

Fortunately, this is easy to do using compile functions. We can add compile functions to pages and questions by passing `compile=my_function`. Compile functions are called immediately before a page is displayed to the user. The compile function's first argument is always its "parent" - the page or question associated with it.

In the example below, we define a function `foo` and then pass it to a `Label` using `Label(compile=foo)`.

Run the cell below and notice that `foo` is called *after* the user enters their name.

In [ ]:
def seed():
    return [
        Page(
            Input("What's your name?")
        ),
        Page(
            # this Label is passed to foo as "label"
            Label("Greeting label.", compile=foo)
        )
    ]


def foo(label):
    print(f"foo was called with parent {label}.")


user = User.make_test_user(seed)
_ = user.test_request(["Aristotle"])

Compile functions are most useful when combined with *partial functions*. `partial` takes a function as its first argument and returns a partial function. The rest of the arguments and keyword arguments passed to `partial` are passed to the "original" function when we call it.

In the example below, we define a function `foo` which takes two arguments: `argument0` and `argument1`. We then create a partial function `bar`. When we call `bar`, it will execute `foo(0, argument1=1)`.

In [ ]:
def foo(argument0, argument1):
    print("argument0 is", argument0, "and argument 1 is", argument1)


bar = partial(foo, 0, argument1=1)
bar

In [ ]:
bar()

The example below uses a "partial compile function" to implement our greeting.

Note that, in the `greet_user` function, `greet_label` is a `Label` object. `greet_label.label` is the attribute of `greet_label` that stores the text displayed to the user.

In [ ]:
def seed():
    return [
        Page(
            name_input := Input("What's your name?")
        ),
        Page(
            # this Label is passed to greet_user as "greet_label"
            Label(compile=partial(greet_user, name_input))
        )
    ]


def greet_user(greet_label, name_input):
    greet_label.label = f"Hello, {name_input.response}!"


user = User.make_test_user(seed)
user.test_request(["Aristotle"]).display()

There are some predefined compile functions we can use as well.

For example, image we want to ask the user who their favorite artist is on page 1. If they go back to page 0 and then forward again to page 1, we want page 1 to be blank. However, Hemlock's default is to save the user's page 1 responses when they go back and forth.

Run the cell below to see how this works.

In [ ]:
def seed():
    return [
        Page(
            Label("Hello, world!")
        ),
        Page(
            Input("Who's your favorite artist?"),
            back=True
        ),
        Page(
            Label("Goodbye!")
        )
    ]


user = User.make_test_user(seed)
# go to page 1
user.test_request()
# fill in "Monet" and then go back to page 0
user.test_request(["Monet"], direction="back")
# go forward to page 1 and notice that "Monet" is filled in on page 1
user.test_request().display()

We can use `compile=compile.clear_response()` to clear the page 1 responses every time the user goes back and forth between pages 0 and 1.

In [ ]:
def seed():
    return [
        Page(
            Label("Hello, world!")
        ),
        Page(
            Input(
                "Who's your favorite artist?",
                compile=compile.clear_response()
            ),
            back=True
        ),
        Page(
            Label("Goodbye!")
        )
    ]


user = User.make_test_user(seed)
user.test_request()
user.test_request(["Monet"], direction="back")
user.test_request().display()

## Exercises

0. Create a survey with 2 pages:

    0. A page with:

        0. A `Check` question asking the user if they made a new year's resolution to lose weight (the choices are "Yes" and "No").
        1. An `Input` question asking, if so, how many pounds the user wants to lose.

    1. A page with a `Label`.
1. Add a compile function that changes the label on page 1 based on the user's responses to page 0:

    0. If the user resolved to lose weight, the label should read, "Good luck reaching your goal of losing XX pounds!"
    1. If the user didn't resolve to lose weight, the label should read, "Congratulations on being at your target weight!"
2. Make a test user. On page 0, make the test user say that they resolved to lose 10 pounds. Check that the label on page 1 reads, "Good luck reaching your goal of losing 10 pounds!"
3. Make another test user. On page 0, make the test user say that they didn't resolve to lose weight. Check that the label on page 1 reads, "Congratulations on being at your target weight!"
4. Transfer the seed function you wrote in steps 0-1 to `src/my_survey.py`, run the app, and repeat the exercise from steps 2-3 in your browser.
5. Test your code with `make test`

In [ ]:
# WRITE YOUR CODE HERE

## Answers

In [ ]:
def seed():
    return [
        Page(
            resolved_to_lose_weight := Check(
                "Did you make a new year's resolution to lose weight?",
                [(1, "Yes"), (0, "No")]
            ),
            weight_loss_amount := Input(
                "If so, how many pounds do you want to lose?",
                input_tag={"type": "number", "min": 0}
            )
        ),
        Page(
            Label(
                compile=partial(make_label, resolved_to_lose_weight, weight_loss_amount)
            )
        )
    ]


def make_label(input, resolved_to_lose_weight, weight_loss_amount):
    if resolved_to_lose_weight.response:
        input.label = f"Good luck reaching your goal of losing {weight_loss_amount.response} pounds!"
    else:
        input.label = "Congratulations on being at your target weight!"


user = User.make_test_user(seed)
user.test_request([1, 10]).display()

In [ ]:
user = User.make_test_user(seed)
user.test_request([0, None]).display()

See `src/compile.py` for what your survey file should look like.

Now you know how to use compile functions! Check out `070_validate.ipynb` for the next part of the tutorial.